In [1]:
import ccxt
import time
import os
import datetime
import pandas as pd

msec = 1000
minute = 60 * msec
hour = 60 * minute

symbol = 'BTC/USDT'
symbol_ = 'BTC_USDT'
market = 'binance'
timewindow = '1h'

if timewindow == '1h':
    offset = hour
    delay = offset / 1000
elif timewindow == '1m':
    offset = minute
    delay = offset / 1000

In [2]:
def get_file_contents(filename):
    try:
        with open(filename, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


binance = ccxt.binance()
home_path = os.path.expanduser("~")
binance.apiKey = get_file_contents(home_path + '/api_key/binance/pub')
binance.secret = get_file_contents(home_path + '/api_key/binance/private')

In [3]:
binance = ccxt.binance({
    'apiKey': binance.apiKey,
    'secret': binance.secret,
})
exchange = binance

In [4]:
from_datetime = '2019-09-29 12:00:00'
from_timestamp = exchange.parse8601(from_datetime)

In [5]:
t = exchange.fetch_trades(symbol=symbol, since=from_timestamp)

In [6]:
t[0]

{'info': {'a': 166647503,
  'p': '8073.86000000',
  'q': '0.02810800',
  'f': 184419896,
  'l': 184419896,
  'T': 1569758400471,
  'm': True,
  'M': True},
 'timestamp': 1569758400471,
 'datetime': '2019-09-29T12:00:00.471Z',
 'symbol': 'BTC/USDT',
 'id': '166647503',
 'order': None,
 'type': None,
 'takerOrMaker': None,
 'side': 'sell',
 'price': 8073.86,
 'amount': 0.028108,
 'cost': 226.94005688,
 'fee': None}

In [16]:
t[0].get('price')

8073.86

In [17]:
t[1].get('price')

8074.05

In [7]:
ohlcvs = exchange.fetch_ohlcv(symbol, timewindow, from_timestamp, limit=5)
df_ohlcvs = pd.DataFrame(ohlcvs, columns=['ts','Open', 'High', 'Low', 'Close', 'Volume'])
df_ohlcvs

,ts,Open,High,Low,Close,Volume
0,1569758400000,8073.86,8078.0,7908.08,7995.96,4237.052032
1,1569762000000,7995.02,8050.0,7965.43,7995.67,1675.788207
2,1569765600000,7996.23,8039.0,7963.36,7971.15,1054.711486
3,1569769200000,7969.31,7990.0,7890.00,7965.57,2058.778764
4,1569772800000,7965.56,8035.0,7926.37,8030.75,1942.000863


In [8]:
ohlcvs = exchange.fetch_ohlcv(symbol, timewindow, from_timestamp) #, limit=1)

In [9]:
        # aggregate trades
        #
        #     [
        #         {
        #             "a": 26129,         # Aggregate tradeId
        #             "p": "0.01633102",  # Price
        #             "q": "4.70443515",  # Quantity
        #             "f": 27781,         # First tradeId
        #             "l": 27781,         # Last tradeId
        #             "T": 1498793709153,  # Timestamp
        #             "m": True,          # Was the buyer the maker?
        #             "M": True           # Was the trade the best price match?
        #         }
        #     ]
        #

In [10]:
t[0]

{'info': {'a': 166647503,
  'p': '8073.86000000',
  'q': '0.02810800',
  'f': 184419896,
  'l': 184419896,
  'T': 1569758400471,
  'm': True,
  'M': True},
 'timestamp': 1569758400471,
 'datetime': '2019-09-29T12:00:00.471Z',
 'symbol': 'BTC/USDT',
 'id': '166647503',
 'order': None,
 'type': None,
 'takerOrMaker': None,
 'side': 'sell',
 'price': 8073.86,
 'amount': 0.028108,
 'cost': 226.94005688,
 'fee': None}

In [11]:
t[0].get('side')

'sell'

In [12]:
N_buy = sum(x.get('side') == 'buy' for x in t)
N_buy

11218

In [13]:
N_sell = sum(x.get('side') == 'sell' for x in t)
N_sell

14716

In [14]:
amount_buy = sum(x.get('amount') if x.get('side') == 'buy' else 0.0 for x in t)
amount_buy

1534.1363219999919

In [15]:
amount_sell = sum(x.get('amount') if x.get('side') == 'sell' else 0.0 for x in t)
amount_sell

2702.9157099999807